In [25]:
import numpy as np
import pandas as pd
import yfinance as yf

In [26]:
# Sector ETF tickers representing the 11 S&P 500 sectors
sector_tickers = [
    "XLF",  # Financials
    "XLK",  # Technology
    "XLV",  # Health Care
    "XLY",  # Consumer Discretionary
    "XLP",  # Consumer Staples
    "XLE",  # Energy
    "XLI",  # Industrials
    "XLU",  # Utilities
    "XLB",  # Materials
    "XLRE",  # Real Estate
    "XLC",  # Communication Services
]

indices = [
    "^GSPC",  # S&P 500
    "^VIX",  # Volatility Index
]

# Date range from the paper
start_date = "2006-01-01"
end_date = "2021-12-31"

In [27]:
# Download daily adjusted close prices for sector ETFs
prices = yf.download(
    sector_tickers + indices,
    start=start_date,
    end=end_date,
    interval="1d",
    # group_by='ticker',
    auto_adjust=True,
    progress=True,
)["Close"]

# adjusted means stock prices have been corrected for corporate actions etc

[*********************100%***********************]  13 of 13 completed


In [28]:
# Print NaN situation
nan_counts = prices.isna().sum()
print("NaN counts before cleaning:")
print(nan_counts[nan_counts > 0])

# XLC was created June 2018
# XLRE was created Oct 2015 ( as split off from XLF )

# Cant really "clean" since the data is simply missing
# prices_clean = prices.copy()

# Replace any missing data using forward fill
# prices_clean = prices_clean.ffill()

# Drop columns with all NaN values
# prices_clean = prices_clean.dropna(axis=1, how='all')

# Print dropped columns
# dropped_cols = list(prices.columns.difference(prices_clean.columns))
# if dropped_cols:
#     print("Dropped columns (all NaN):", dropped_cols)

# prices_clean

NaN counts before cleaning:
Ticker
XLC     3136
XLRE    2458
dtype: int64


In [29]:
# Compute log returns for sector prices
log_returns = np.log(prices / prices.shift(1))

# Drop the first row which will have NaNs
log_returns = log_returns.dropna(axis=0, how="all")

# Drop VIX col, since we dont need return of volatility
log_returns = log_returns.drop(columns=["^VIX"])

log_returns

Ticker,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,^GSPC
Date,,,,,,,,,,,,
2006-01-04,0.006127,NaN,0.002842,-0.000933,0.002526,0.011663,0.003407,NaN,0.001561,0.009916,0.000909,0.003666
2006-01-05,-0.001609,NaN,-0.012759,0.003414,-0.001578,0.004628,-0.005115,NaN,-0.006574,-0.002470,0.000907,0.000016
2006-01-06,0.010568,NaN,0.024234,0.005869,0.003783,0.015575,0.004264,NaN,0.010622,0.006471,0.008134,0.009356
2006-01-09,-0.000319,NaN,-0.001123,0.002768,0.008148,0.003176,0.004670,NaN,-0.006235,0.004596,0.010150,0.003650
2006-01-10,-0.006394,NaN,0.010803,0.000921,-0.002813,-0.000907,-0.002545,NaN,0.001563,-0.003675,0.000297,-0.000357
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,0.010015,0.008873,0.000547,0.005166,0.011712,0.005567,0.002261,-0.003000,-0.000144,0.004101,0.013776,0.006204
2021-12-27,0.014948,0.009683,0.021623,0.010251,0.010529,0.021226,0.010575,0.020227,0.004865,0.010497,0.009697,0.013744
2021-12-28,0.005784,-0.000507,-0.000713,0.000510,0.005034,-0.005222,0.005638,0.004310,0.008952,-0.002632,0.000439,-0.001011


In [30]:
# calc vol metrics based on paper
sp500_returns = prices["^GSPC"].pct_change()  # simple returns
vol20 = sp500_returns.rolling(20).std()
vol60 = sp500_returns.rolling(60).std()
vol_ratio = vol20 / vol60

# create df to hold vol metrics
vol_df = pd.DataFrame(
    {
        "vol20": vol20,
        "vol60": vol60,
        "vol_ratio": vol_ratio,
        "VIX": prices["^VIX"],
    }
)
# drop first 60 rows
# vol_df = vol_df.dropna()

# Standardize the metrics using expanding lookback window to prevent look-ahead bias
# note that expanding uses all past data up to the current row
for col in ['vol20', 'vol60', 'vol_ratio', 'VIX']:
    mean = vol_df[col].expanding().mean()
    std = vol_df[col].expanding().std()
    vol_df[f'{col}_standard'] = (vol_df[col] - mean) / std

# note that for any two values x_1 and x_2, the standardized value of the second one is always:
# \pm \frac{1}{\sqrt{2}} \approx \pm 0.7071
# which is why the first standardized value is always 0.7071 ( for positive values )

# Drop the first row with NaN since there is no std yet
# vol_df = vol_df.dropna()
vol_df

,vol20,vol60,vol_ratio,VIX,vol20_standard,vol60_standard,vol_ratio_standard,VIX_standard
Date,,,,,,,,
2006-01-03,NaN,NaN,NaN,11.140000,NaN,NaN,NaN,NaN
2006-01-04,NaN,NaN,NaN,11.370000,NaN,NaN,NaN,0.707107
2006-01-05,NaN,NaN,NaN,11.310000,NaN,NaN,NaN,0.307341
2006-01-06,NaN,NaN,NaN,11.000000,NaN,NaN,NaN,-1.221480
2006-01-09,NaN,NaN,NaN,11.130000,NaN,NaN,NaN,-0.402241
...,...,...,...,...,...,...,...,...
2021-12-23,0.013290,0.009031,1.471538,17.959999,0.417919,-0.213812,1.879961,-0.166061
2021-12-27,0.012438,0.009001,1.381847,17.680000,0.305785,-0.218192,1.542242,-0.195961
2021-12-28,0.012177,0.008917,1.365616,17.540001,0.271549,-0.230123,1.480663,-0.210898


In [31]:
# Join vol_df with log_returns on index (Date)
final_df = log_returns.join(vol_df[['vol20_standard', 'vol_ratio_standard', 'VIX_standard']])
final_df

,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,^GSPC,vol20_standard,vol_ratio_standard,VIX_standard
Date,,,,,,,,,,,,,,,
2006-01-04,0.006127,NaN,0.002842,-0.000933,0.002526,0.011663,0.003407,NaN,0.001561,0.009916,0.000909,0.003666,NaN,NaN,0.707107
2006-01-05,-0.001609,NaN,-0.012759,0.003414,-0.001578,0.004628,-0.005115,NaN,-0.006574,-0.002470,0.000907,0.000016,NaN,NaN,0.307341
2006-01-06,0.010568,NaN,0.024234,0.005869,0.003783,0.015575,0.004264,NaN,0.010622,0.006471,0.008134,0.009356,NaN,NaN,-1.221480
2006-01-09,-0.000319,NaN,-0.001123,0.002768,0.008148,0.003176,0.004670,NaN,-0.006235,0.004596,0.010150,0.003650,NaN,NaN,-0.402241
2006-01-10,-0.006394,NaN,0.010803,0.000921,-0.002813,-0.000907,-0.002545,NaN,0.001563,-0.003675,0.000297,-0.000357,NaN,NaN,-1.450386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23,0.010015,0.008873,0.000547,0.005166,0.011712,0.005567,0.002261,-0.003000,-0.000144,0.004101,0.013776,0.006204,0.417919,1.879961,-0.166061
2021-12-27,0.014948,0.009683,0.021623,0.010251,0.010529,0.021226,0.010575,0.020227,0.004865,0.010497,0.009697,0.013744,0.305785,1.542242,-0.195961
2021-12-28,0.005784,-0.000507,-0.000713,0.000510,0.005034,-0.005222,0.005638,0.004310,0.008952,-0.002632,0.000439,-0.001011,0.271549,1.480663,-0.210898


In [32]:
# Save dataframe to csv
final_df.to_csv('../data/train.csv', index=True)